In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import copy
import random
from tqdm import tqdm
from collections import Counter

In [3]:
import warnings
warnings.filterwarnings('ignore') # setting ignore as a parameter

In [4]:
# 人設（性格）
personalities = [
    "非常堅持己見的固執（完全不願意聽別人講或不願意承認錯誤）",  # 0.2
    "會理性論述的固執（會努力想盡辦法說服對方）",  # 0.25
    "願意傾聽溝通（會聽別人講且會妥協）",  # 0.25
    "具有同理心的好好先生（很替別人著想，但會試圖多了解原因或陳述需求）",  # 0.2
    "喜歡迂迴、常常不正面處理問題（想透過各種無厘頭的方式為自己謀取利益 e.g. 拉關係）（若角色的任務不適合該性格，則可以自由更換性格）"  # 0.1
]

In [5]:
scenes = [
    # Case 1
    {"場景":"在校園的某處，「同學A」對「同學B」説...",
     "語者一":{
         "角色":"語者一：同學A",
         "任務":"課業或工作上發生了某件事（自由發揮 e.g.考試搞砸、分組踩雷、被老闆罵...），你正在向同學B抱怨吐苦水",
     },
     "語者二":{
         "角色":"語者二：同學B",
         "任務":"陪伴並試圖努力安慰對方",
     },
    },
    ###############################################################################################
    # Case 2
    {"場景":"在校園的某處，「同學A」對「同學B」説...",
     "語者一":{
         "角色":"語者一：同學A",
         "任務":"感情或家庭層面發生了某件事（自由發揮 e.g.失戀、朋友吵架、家人關係不好...），你正在向同學B抱怨吐苦水",
     },
     "語者二":{
         "角色":"語者二：同學B",
         "任務":"陪伴並試圖努力安慰對方",
     },
    },
    ###############################################################################################
    ###############################################################################################
    # Case 3
    {"場景":"在悠閒的假日市集，路過的遊客向攤位的賣家提出了一個要求...",
     "語者一":{
         "角色":"語者一：路過的遊客",
         "任務":"想購買攤位的某個東西（自由發揮 e.g.衣服、零食、生活小物...），但認為價格太貴，因此試圖想跟賣家殺價",
     },
     "語者二":{
         "角色":"語者二：攤位的賣家",
         "任務":"提出各種原因（自由發揮 e.g.小本經營、已經有折扣、強調商品特殊性...），試圖以原本的售價賣出商品",
     },
    },
    ###############################################################################################
    # Case 4
    {"場景":"在網路購物平台上，客戶向賣家提出了一個要求...",
     "語者一":{
         "角色":"語者一：網購客戶",
         "任務":"因為某些原因（自由發揮 e.g.商品瑕疵、尺寸不合...）想要退貨，但不想自己負擔運費",
     },
     "語者二":{
         "角色":"語者二：網購賣家",
         "任務":"提出各種原因（自由發揮 e.g.瑕疵在可接受範圍、當初有講好...），認為退貨運費應該由客戶負擔",
     },
    },
    ###############################################################################################
    # Case 5
    {"場景":"在網路購物平台上，客戶向賣家提出了一個要求...",
     "語者一":{
         "角色":"語者一：網購客戶",
         "任務":"想購買某個東西（自由發揮 e.g.衣服、零食、球鞋、生活小物...），但你發現折扣活動到昨天剛結束，因此你希望仍然能夠凹到折扣價",    },
     "語者二":{
         "角色":"語者二：網購賣家",
         "任務":"想辦法提出各種理由（自由發揮），試圖以原本的售價賣出商品",
     },
    },
    ###############################################################################################
    ###############################################################################################
    # Case 6
    {"場景":"在客服平台上，客戶提出了一個問題...",
     "語者一":{
         "角色":"語者一：網購客戶",
         "任務":"基於某些原因（自由發揮 e.g.商品有瑕疵、延遲到貨...），因此你希望能夠爭取到最多的補償",
     },
     "語者二":{
         "角色":"語者二：客服人員",
         "任務":"提出各種原因（自由發揮 e.g.瑕疵在可接受範圍、合約上有寫...），試圖拒絕對方的補償請求",
     },
    },
    ###############################################################################################
    # Case 7
    {"場景":"在客服平台上，客戶提出了一個問題...",
     "語者一":{
         "角色":"語者一：網購客戶",
         "任務":"你前幾天剛下訂了某樣商品（自由發揮 e.g.衣服、零食、球鞋、生活小物...），但今天你發現這項商品竟然降價了，因此你希望對方能夠補給你差額",
     },
     "語者二":{
         "角色":"語者二：客服人員",
         "任務":"提出各種原因（自由發揮 e.g.公司規定...），試圖拒絕補償對方",
     },
    },
    ###############################################################################################
    # Case 8
    {"場景":"餐廳的電話響起，是一名顧客的來電...",
     "語者一":{
         "角色":"語者一：顧客",
         "任務":"你想訂這間餐廳在這週末的位置，基於某些原因（自由發揮 e.g.家人生日、朋友隔天就要出國...），你非常迫切需要訂到位",
     },
     "語者二":{
         "角色":"語者二：餐廳服務生",
         "任務":"你發現餐廳這週末的訂位已經客滿，因此試圖拒絕顧客訂位的要求",
     },
    },
    ###############################################################################################
    # Case 9
    {"場景":"在餐廳中，餐廳經理被顧客呼喚過來，顧客對餐廳經理說...",
     "語者一":{
         "角色":"語者一：在餐廳用餐的顧客",
         "任務":"基於某些原因（自由發揮 e.g.上菜太慢、服務不佳、餐點有問題、小孩跌倒...），因此你希望能夠爭取到最多的補償",
     },
     "語者二":{
         "角色":"語者二：餐廳經理",
         "任務":"解釋責任歸屬不在自己，希望沒有損害賠償 or 以例外事件安撫，希望沒有賠償或試圖降低賠償的程度 or 承認錯誤，願意賠償但試圖降低賠償的損失（根據角色性格，自由選擇一個任務）",
     },
    },          
    ###############################################################################################
    # Case 10
    {"場景":"在旅館櫃檯，房務經理被顧客呼喚過來，顧客對房務經理說...",
     "語者一":{
         "角色":"語者一：前一晚住房的房客",
         "任務":"基於某些原因（自由發揮 e.g.晚上沒有熱水、房間很髒、隔壁很吵...），因此你希望能夠爭取到最多的補償",  
     },
     "語者二":{
         "角色":"語者二：房務經理",
         "任務":"解釋責任歸屬不在自己，希望沒有損害賠償 or 以例外事件安撫，希望沒有賠償或試圖降低賠償的程度 or 承認錯誤，願意賠償但試圖降低賠償的損失（上述三者，根據角色性格，可以自由選擇一個任務）",
     },
    },
    ###############################################################################################
    ###############################################################################################
    # Case 11
    {"場景":"在百貨公司的櫃位前，銷售人員對經過的顧客說...",
     "語者一":{
         "角色":"語者一：銷售人員",
         "任務":"推薦某項商品與優惠方案（自由發揮 e.g.保養品買一送一、週年慶全館一折...），試圖盡最大努力達成交易",  
     },
     "語者二":{
         "角色":"語者二：路過的顧客",
         "任務":"對商品完全沒有需求 or 對商品有需求但不急迫 or 對商品有需求但想用最便宜的價格購入（上述三者，根據角色性格，可以自由選擇一個任務）",
     },
    },
    ###############################################################################################
    ###############################################################################################
    # Case 12
    {"場景":"在悠閒的假日午後，「同學A」對「同學B」説...",
     "語者一":{
         "角色":"語者一：同學A",
         "任務":"基於某些原因（自由發揮 e.g.有優惠券、很久沒見面...），因此想邀約同學B一起去某個地方玩耍（自由發揮 e.g.吃飯、打球、看電影...）",  
     },
     "語者二":{
         "角色":"語者二：同學B",
         "任務":"基於某些原因（自由發揮 e.g.手上有急事、很想睡覺...），不太想跟對方出門，因此試圖拒絕對方的邀約",
     },
    },
    ###############################################################################################
    # Case 13
    {"場景":"討論到下週末的旅行，「同學A」對「同學B」説...",
     "語者一":{
         "角色":"語者一：同學A",
         "任務":"想去某個景點 or 想吃某個東西 or 想搭某種交通工具（或其他自由發揮）",  
     },
     "語者二":{
         "角色":"語者二：同學B",
         "任務":"基於某些原因（自由發揮 e.g.自己不喜歡、很貴、很遠不方便...），試圖拒絕對方的提案",
     },
    },
    ###############################################################################################
    # Case 14
    {"場景":"在餐廳櫃檯，正在處理關於訂位的事情，服務生對顧客說...",
     "語者一":{
         "角色":"語者一：服務生",
         "任務":"發現系統查不到該顧客的訂位資料，因此試圖向顧客解釋為何他的訂位沒有成功（自由發揮 e.g.顧客訂錯時間、昨天打電話確認無人接聽、顧客自己按到取消...）",
     },
     "語者二":{
         "角色":"語者二：顧客",
         "任務":"試圖替自己爭取權益，希望仍然能在今天用餐",
     },
    },          
    ###############################################################################################
]

In [6]:
def generate_prompts(total_dialogue_count):
    personalities_random_list = random.choices(personalities, weights=(0.2, 0.25, 0.25, 0.2, 0.1), k=2*total_dialogue_count)
    print(Counter(personalities_random_list))
    prompts = []
    for i in range(total_dialogue_count):
        # scene_idx = i % len(scenes)
        scene_idx = random.randint(0, len(scenes)-1)
        p1, p2 = personalities_random_list[i*2], personalities_random_list[(i*2)+1]
        prompt = copy.deepcopy(scenes[scene_idx])
        prompt["語者一"]["性格"] = p1
        prompt["語者二"]["性格"] = p2
        prompts.append(prompt)
    return prompts

In [7]:
def generate_prompt_file(root_path, folder_id, dialogue_id, prompt):
    # Define the roles
    roles = ["host", "guest"]
    # Define and shuffle the order of speakers
    speakers = ["語者一", "語者二"]
    random.shuffle(speakers)
    # Write to the txt files
    for role, speaker in zip(roles, speakers):
        # Construct folder path
        folder_path = os.path.join(root_path, folder_id)
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)
        # Construct file path
        file_path = os.path.join(folder_path, f"{dialogue_id}-{role}.txt")
        with open(file_path, "w") as file:
            file.write(f"場景: {prompt['場景']}\n")
            file.write(f"角色: {prompt[speaker]['角色']}\n")
            file.write(f"任務: {prompt[speaker]['任務']}\n")
            file.write(f"性格: {prompt[speaker]['性格']}\n")

In [8]:
def generate_all_prompt_files(root_path, total_dialogue_count):
    if not os.path.exists(root_path):
        os.makedirs(root_path)
    dialogue_ids = [f"dialogue-{str(i).zfill(4)}" for i in range(0, total_dialogue_count)]
    folder_ids = [f"dialogue {str(i).zfill(4)}-{str(i+99).zfill(4)}" for i in range(0, total_dialogue_count, 100)]
    prompts = generate_prompts(total_dialogue_count)
    assert len(prompts) == len(dialogue_ids)
    for i in tqdm(range(len(folder_ids))):
        for j in range(i*100, i*100+100):
            folder_id = folder_ids[i]
            dialogue_id = dialogue_ids[j]
            prompt = prompts[j]
            # Generate files
            generate_prompt_file(root_path, folder_id, dialogue_id, prompt)
            # Only print the first and last file name in each folder
            if j == i*100 or j == i*100+99:
                print(folder_id, dialogue_id)

In [9]:
# Main
root_path = "/content/drive/MyDrive/論文資料完整版/prompts"
total_dialogue_count = 2000
generate_all_prompt_files(root_path, total_dialogue_count)

Counter({'會理性論述的固執（會努力想盡辦法說服對方）': 996, '願意傾聽溝通（會聽別人講且會妥協）': 927, '具有同理心的好好先生（很替別人著想，但會試圖多了解原因或陳述需求）': 886, '非常堅持己見的固執（完全不願意聽別人講或不願意承認錯誤）': 827, '喜歡迂迴、常常不正面處理問題（想透過各種無厘頭的方式為自己謀取利益 e.g. 拉關係）（若角色的任務不適合該性格，則可以自由更換性格）': 364})


  0%|          | 0/20 [00:00<?, ?it/s]

dialogue 0000-0099 dialogue-0000


  5%|▌         | 1/20 [00:01<00:20,  1.07s/it]

dialogue 0000-0099 dialogue-0099
dialogue 0100-0199 dialogue-0100


...

dialogue 1700-1799 dialogue-1799
dialogue 1800-1899 dialogue-1800


 95%|█████████▌| 19/20 [00:20<00:01,  1.06s/it]

dialogue 1800-1899 dialogue-1899
dialogue 1900-1999 dialogue-1900


100%|██████████| 20/20 [00:21<00:00,  1.07s/it]

dialogue 1900-1999 dialogue-1999
